# 목차
- [이미지 크롤링](#Image-Crawling)
- [텍스트 크롤링](#PDF의-Text긁어오기)
    - [1. Google Cloud Vision 사용하기](#1.-Google-Cloud-Vision을-활용하는-방법)
    - [2. tika 사용하기](#2.-Tika를-활용하는-방법)

In [1]:
import os
print(os.getcwd())

C:\Users\2021090


# Image Crawling

In [10]:
import time
import os
import sys

# 작업 시작 메시지를 출력합니다.
print("Process Start")

# 시작 시점의 시간을 기록합니다.
start_time = time.time()

# 하나로 합칠 파일들이 저장된 폴더 이름을 시스템으로부터 입력받습니다.
directory = './Patent_PDF/'

# 폴더의 내용물을 열람해 목록을 생성합니다.
filelist = os.listdir(directory)

Process Start


In [11]:
# 캡쳐한 이미지를 저장할 파일 만들기
os.mkdir('./extracted_from_PDF/')

In [12]:
# 필요한 모듈 불러오기
import fitz
import io
from PIL import Image

for pdf_idx, pdf in enumerate(filelist):
    pdf_file = fitz.open('./' + directory + pdf)
    
    page = pdf_file[0] # PDF파일의 맨 첫장만 할당
    image_list = page.getImageList()
    
    for image_index, img in enumerate(image_list[1:3], start=0): # 0번 index는 특허청 마크, 3번 index는 QR코드
        # image의 XREF를 받아옵니다.
        xref = img[0]
        # Image의 byte를 추출합니다.
        base_image = pdf_file.extractImage(xref)
        image_bytes = base_image["image"]
        # Image extension을 받아옵니다.
        image_ext = base_image["ext"]
        # PIL로 load합니다.
        image = Image.open(io.BytesIO(image_bytes))
        # 로컬로 저장합니다
        image.save(open("extracted_from_PDF/extracted_from_{}_{}.png".format(pdf_idx + 1, image_index + 1),"wb"))

# PDF의 Text긁어오기

## 1. Google Cloud Vision을 활용하는 방법

## PDF들의 첫번째, 두번째 페이지를 Image로 긁어오기

In [1]:
from pdf2image import convert_from_path, convert_from_bytes
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)
import matplotlib.pyplot as plt
from PIL import Image as pimg

In [4]:
# patent PDF가 저장된 파일의 목록을 할당
import os
patent_list= os.listdir('./Patent_PDF/')

# 이미지를 저장할 파일 만들기
os.mkdir('./images/')

In [6]:
# 이미지 변환
for patent in patent_list:
    image_patent = convert_from_path(os.path.join('./Patent_PDF/',patent), last_page= 2)

    for idx ,image in zip(range(len(image_patent)),image_patent):
        image.save('./images/patent_{}_{}.png'.format(patent_list.index(patent)+1,idx+1), format='png')
    
    print("{}번째 PDF 이미지추출 완료".format(patent_list.index(patent)+1))

1번째 진행 완료
2번째 진행 완료
3번째 진행 완료
4번째 진행 완료
5번째 진행 완료
6번째 진행 완료
7번째 진행 완료
8번째 진행 완료


### Image에서 Text 가져오기

In [32]:
from google.cloud import vision
import io
import pickle
from tqdm import tqdm

In [33]:
# CREDENTIAL 설정하기
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/2021090/Downloads/feisty-beacon-292207-bf13802b2a35.json"
!echo $GOOGLE_APPLICATION_CREDENTIALS

$GOOGLE_APPLICATION_CREDENTIALS


In [34]:
# 이미지 파일이 저장되어있는 경로로 변경하기
os.chdir('images/')
print(os.getcwd())

C:\Users\2021090\crawling\20210428\images


In [35]:
# path를 받아 text를 잡아내는 함수
def detect_text(path):
    client = vision.ImageAnnotatorClient()
    
    with io.open(path, 'rb') as image_file:
        content = image_file.read()
        image = vision.Image(content = content)
        response = client.text_detection(image=image)
        texts = response.text_annotations
        
        print("Texts: ")
        
        for text in texts:
            print('\n"{}'.format(text.description))
            vertices = (['({},{})'.format(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices])
            
            print('bounds: {}'.format(','.join(vertices)))
            if response.error.message:
                raise Exception(
                '{}\nFor more info on error messages, check: '
                    'https://cloud.google.com/apis/design/errors'.format(response.error.message)
                )
                
                return response, texts

In [36]:
# 이미지 파일 경로
img_files = []

for root_dir, sub_dir, files in os.walk('C:/Users/2021090/crawling/20210428/images/'):
    for fname in files:
        img_files.append(os.path.join(root_dir, fname))
        
img_files[:10]

['C:/Users/2021090/crawling/20210428/images/patent_1_1.png',
 'C:/Users/2021090/crawling/20210428/images/patent_1_2.png',
 'C:/Users/2021090/crawling/20210428/images/patent_2_1.png',
 'C:/Users/2021090/crawling/20210428/images/patent_2_2.png',
 'C:/Users/2021090/crawling/20210428/images/patent_3_1.png',
 'C:/Users/2021090/crawling/20210428/images/patent_3_2.png',
 'C:/Users/2021090/crawling/20210428/images/patent_4_1.png',
 'C:/Users/2021090/crawling/20210428/images/patent_4_2.png',
 'C:/Users/2021090/crawling/20210428/images/patent_5_1.png',
 'C:/Users/2021090/crawling/20210428/images/patent_5_2.png']

In [37]:
# 실행 전 유의사항
# 1. google cloud platform에서 발급한 json파일이 유효해야함
# 2. 회사 네트워크로는 연결 불가. 개인 핫스팟 사용 필요
test_image = "patent_4_1.png"
response, texts = detect_text(test_image)
texts[0].description()

Texts: 

"공개특허 10-2019-0096944
(19) 대한민국특허청 (KR)
(12) 공개특허공보(A)
(11) 공개번호
10-2019-0096944
(43) 공개일자
2019년08월20일
(71) 출원인
가꼬우 호징 관세이 가쿠잉
일본국 효고켄 니시노미야시 우에가하라이치반쵸
(51) 국제특허분류(Int. Cl.)
CO9K 11/06 (2006.01)
HOIL 51/00 (2006.01) HO1L 51/50 (2006.01)
CO7F 5/02 (2006.01)
(52) CPC특허분류
1-155
CO7F 5/027 (2013.01)
제이엔씨 주식회사
CO9K 11/06 (2013.01)
(21) 출원번호
일본 도쿄도 치요다쿠 오테마치 2쵸메 2반 1고
(72) 발명자
10-2019-7011911
(22) 출원일자(국제) 2017년12월11일
하타케야마 다쿠지
심사청구일자 없음
(85) 번역문제출일자 2019년04월24일
(86) 국제출원번호 PCT/JP2017/044361
WO 2018/110497
2018년06월21일
일본 효고켄 산다시 가쿠엔 2쵸메 1반치 관세이
가쿠잉 다이가쿠 리코가쿠부내
왕 구오광
일본 지바켄 이치하라시 고이카이간 5-1 제이엔씨
석유 화학 주식회사 이치하라 겐큐쇼내
(87) 국제공개번호
국제공개일자
(30) 우선권주장
사사다 야스유키
일본 지바켄 이치하라시 고이카이간 5-1 제이엔씨
석유 화학 주식회사 이치하라 겐큐쇼내
(74) 대리인
유미특허법인
JP-P-2016-243946 2016년12월 16일 일본(JP)
전체 청구항 수 : 총 18 항
(54) 발명의 명칭 다환방향족 아미노 화합물
(57) 요 약
예를 들면 하기 화학식과 같은 아미노 치환기를 가지는, 붕소 원자와 산소 원자 등으로 복수의 방향족환을 연결
한 신규한 다환방향족 아미노 화합물을 제공함으로써, 유기 EL 소자용 재료의 선택지를 증가시킨다. 또한 신규한
다환 방향족 아미노 화합물을 유기 전계 발광 소자용 재료로서 사용함으로써, 우수한 유기 EL 소자를 제공

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
text_list = []
for img in img_files:
    response, texts = detect_text(img)
    
    if len(texts) == 0:
        continue
    
    else:
        text_list.append(texts[0].description)

## 2. Tika를 활용하는 방법

In [44]:
# 디렉터리 다시 변경
os.chdir('C:/Users/2021090/crawling/20210428/')

In [45]:
from tika import parser

In [48]:
test_file = './Patent_PDF/1020197009381.pdf'

# Parse data from file
file_data = parser.from_file(test_file, service='text')

# Get files text content
text = file_data['content'].strip()

text = str(text)
safe_text = text.encode('euc-kr', errors='ignore')
print(safe_text.decode('euc-kr'))

(19) 민 특허청(KR)
(12) 공개특허공보(A)

(11) 공개    10-2019-0051003

(43) 공개    2019 05월14

(51) 특허 (Int. Cl.)

     C07F 5/02 (2006.01)  C07F 9/6584 (2006.01)

     C09K 11/06 (2006.01)  F21S 2/00 (2016.01)

     G09F 9/30 (2006.01)  H01L 29/786 (2006.01)

     H01L 51/00 (2006.01)  H01L 51/05 (2006.01)

     H01L 51/50 (2006.01)

(52) CPC특허

     C07F 5/027 (2013.01)

     C07F 9/65842 (2013.01)
(21) 원        10-2019-7009381

(22) 원 ( ) 2017 08월25

     심사청    없  

(85) 역  2019 04월01

(86) 원    PCT/JP2017/030542

(87) 공개    WO 2018/047639

     공개    2018 03월15

(30) 우 주

JP-P-2016-174208  2016 09월07   본(JP)

(71) 원

가 우 징  가쿠

본  고  니시 미 시 우에가 라 치
1-155

엔  주식 사

본 도쿄도 치 다쿠 마치 2  2  1고

(72) 

타 마 다쿠지

본 고  산다시 가쿠엔 2  1 치 
가쿠  다 가쿠 리 가쿠 내

마쓰  고헤

본 고  산다시 가쿠엔 2  1 치 
가쿠  다 가쿠 리 가쿠 내

(뒷 에 계 )

(74) 리

미특허

체 청   :  22 

(54)  칭 다   

(57)  

질  원   원  등  복   연결  신규  다    공 ,  EL

 재료  택 사  가시 다. 또 , 신규  다     계   재료  사

, 우   EL  공 다.

공개특허 10-2019-0051003

- 1 -



(52) CPC특허

     C09K 11/